In [1]:
# Import libraries
import pandas as pd
import requests
from bs4 import BeautifulSoup 
import json, codecs, os
from time import time

In [2]:
"""
def 01

Description: def parse_html_from_url(url)
- Return html tree structor of a newspaper(or a url)
"""

def parse_html_from_url(url):
    response  = requests.get(url)
    html_tree = BeautifulSoup(response.text, 'lxml')
 
    return html_tree

In [3]:
'''
def 02

Description: get_name_topic(url):
- Return name topic of a url
'''
def get_name_topic(url):
    page_html_tree = parse_html_from_url(url)
    
    try:
        page_name = page_html_tree.find(class_= "breadcrumb left")

        name = ''

        for i in page_name.findAll('a'):

            name += ' ' + i.text
            
            return name

    except:
            return None
  

In [4]:
# Get all data a page, such as: thoi-su-p2
# give: url of page, topic
# return: list has element is dictionary(title, content)
# such as:    data [ {'url': ,'topic': , 'title': , 'content': }, 
#                    {'url': ,'topic': , 'title': , 'content': }
#                  ]

def get_data_from_all_urls_in_a_page(url_page, topic):
    page_html_tree = parse_html_from_url(url_page)
    page_sidebar_1 = page_html_tree.find(class_= "sidebar_1") 
  
    data = []
   
    for url in page_sidebar_1.find_all(class_ = 'title_news'):
        element_url = url.find('a')['href']  # get url
        name = get_name_topic(element_url)
        
        if (name == topic):                            # check topic
            data1 = crawl_data_from_a_url(element_url) # element dictionary
            
            if(data1 != None):
                data.append(data1)                     # add list data

    return data

In [5]:
"""
Description: def crawl_data_from_a_url(url)
- Get title and content of a newspaper(it is a url)
- Return a dictionary has kind 
       dic {'Title': title,
            'Content': content}
"""

def crawl_data_from_a_url(url):
    data = {}  
    html_tree = parse_html_from_url(url)
    
    try: 
        element_sidebar_1 = html_tree.find(class_= "sidebar_1")
        
         # 1. Get topic
        nameTopic = get_name_topic(url)
        nameTopic = nameTopic.strip()

        # 2. Get title
        title = element_sidebar_1.find(class_= "title_news_detail mb10").text
        title = title.replace('\n', '')
        title = title.replace('\t', '')
        title = title.strip()
        
        # 3. Get content
        content = ''
      
        for paragraph in element_sidebar_1.find_all(class_ = 'Normal'):
            content += ' ' + paragraph.text
      
        content = content.replace('\n','')
        content = content.replace('\t','')
        content = content.strip()
 
        # 4. Get into data
        if(len(url) < 2):
            data.update({'Link': 'NULL'})
        else:
            data.update({'Link': url})
            
        if(len(nameTopic) < 2):
            data.update({'Topic': 'NULL'}) 
        else: 
            data.update({'Topic': nameTopic})
            
        if(len(title) < 2):
            data.update({'Title': 'NULL'})
        else:
            data.update({'Title': title})
        
        if(len(content) < 2):
            data.update({'Content': 'NULL'})
        else:
            data.update({'Content': content})

        return data 
  
    except:
        return None

In [6]:
def save_data_of_a_page_to_txt(listData, fileName):
    f = codecs.open(fileName, 'a+', 'utf-8')
    
    for i in listData:
        
        f.write(i['Link'] + '\t')
        f.write(i['Topic'] + '\t')  
        f.write(i['Title'] + '\t')
        f.write(i['Content'])
        f.write('\n')

    f.close()

    return None

In [7]:
# Get topic name from page 1 of topic
def get_topic(page_urls):
    topic = []
    k = 1
    
    for page in page_urls:
        nameTopic = get_name_topic(page)
        topic.insert(k, nameTopic)
        k += 1
        
    return topic

In [8]:
'''
give: topic, list page_url
return: data all page of page_url
'''
def get_all_data_of_topic(topic, page_url):
    number = 2041
    temp = []
  
    while(number <= 2060):
        page = page_url + '-p' + str(number)
        temp += get_data_from_all_urls_in_a_page(page, topic)
        number += 1
  
    return temp, number

In [9]:

def data_Collection(page_urls):
    topic = get_topic(page_urls)
    k = 0
    data = []
  
    # Get data
    for i in page_urls:
        temp, number = get_all_data_of_topic(topic[k], i)    
        data += temp
        k += 1
        
    fileInput = 'Input.txt'
    save_data_of_a_page_to_txt(data, fileInput)

    return data, fileInput

In [ ]:
# Main collect data 
start = time()

page_urls = ['https://vnexpress.net/thoi-su',
              'https://vnexpress.net/the-gioi',
              'https://vnexpress.net/phap-luat', 
              'https://vnexpress.net/khoa-hoc',
              'https://vnexpress.net/giao-duc']

data, fileIn = data_Collection(page_urls)

print(len(data))

end = time() - start

print(end)

In [ ]:
df = pd.read_csv("Input.txt", header = None, sep = "\t", encoding = "utf-8")

In [ ]:
len(df)

In [ ]:
df.isnull().sum().sum()